In [1]:
import pandas as pd

df = pd.read_csv("train.csv")
df.head()

,text,label
0,— Кровь! какую кровь? — встревожилась,1
1,– Под нижнюю подушку.,0
2,— Благодарю-с...,1
3,— Когда же это-с?,1
4,"Старуха помолчала, как бы в раздумье,",1


In [2]:
from sklearn.model_selection import train_test_split

train_texts, valid_texts, train_labels, valid_labels = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42, stratify=df["label"])

In [3]:
from datasets import Dataset

train = Dataset.from_dict({"text": train_texts, "label": train_labels})
valid = Dataset.from_dict({"text": valid_texts, "label": valid_labels})

In [4]:
from transformers import AutoTokenizer

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

train = train.map(tokenize, batched=True)
valid = valid.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/4512 [00:00<?, ? examples/s]

Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilroberta-base", num_labels=2)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00


In [7]:
import numpy as np
import evaluate

metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = metric_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": acc, "f1": f1}

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="classifier",
    eval_strategy="epoch",
    push_to_hub=False,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    metric_for_best_model="accuracy"
)

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=valid,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: artem-vgk (artem-vgk-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.306000,0.227403,0.891844,0.892982
2,0.238400,0.239325,0.897163,0.901695
3,0.234000,0.216880,0.908688,0.911436


TrainOutput(global_step=1692, training_loss=0.25414629410908296, metrics={'train_runtime': 825.4002, 'train_samples_per_second': 16.399, 'train_steps_per_second': 2.05, 'total_flos': 1793078708207616.0, 'train_loss': 0.25414629410908296, 'epoch': 3.0})

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

pred = trainer.predict(valid)
logits, labels = pred.predictions, pred.label_ids
preds = np.argmax(logits, axis=-1)

acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds)
cm = confusion_matrix(labels, preds)

print("accuracy_score:", acc)
print("f1_score:", f1)
print("confusion_matrix:", cm)

accuracy_score: 0.9086879432624113
f1_score: 0.9114359415305245
confusion_matrix: [[495  38]
 [ 65 530]]


In [13]:
import re

with open("mlm.txt", "r", encoding="utf-8") as f:
    text = f.read()

text = re.sub(r'[^\w\s]', '', text)

words = text.split()

sent = []
size = 100

for i in range(0, len(words), size):
    chunk = words[i:i + size]
    sentence = ' '.join(chunk)
    sent.append(sentence)

sent = [s.strip() for s in sent]

dataset = Dataset.from_dict({"text": sent})
dataset = dataset.train_test_split(test_size=0.2)

In [15]:
tokenizer_mlm = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

def tokenize_mlm(examples):
    return tokenizer_mlm(examples["text"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(
    tokenize_mlm,
    batched=True,
    num_proc=4,
    remove_columns=["text"]
)

Map (num_proc=4):   0%|          | 0/3030 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/758 [00:00<?, ? examples/s]

In [16]:
block_size = 128

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_len = len(concatenated["input_ids"])
    total_len = (total_len // block_size) * block_size
    return {
        k: [t[i : i + block_size] for i in range(0, total_len, block_size)]
        for k, t in concatenated.items()
    }

lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/3030 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/758 [00:00<?, ? examples/s]

In [17]:
from transformers import DataCollatorForLanguageModeling

tokenizer_mlm.pad_token = tokenizer_mlm.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer_mlm, mlm_probability=0.15)

In [18]:
from transformers import AutoModelForMaskedLM

model_mlm = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base")

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer

training_args_mlm = TrainingArguments(
    output_dir="mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer_mlm = Trainer(
    model=model_mlm,
    args=training_args_mlm,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_mlm,
)


/tmp/ipython-input-4160586370.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_mlm = Trainer(


In [20]:
import math

start_eval = trainer_mlm.evaluate()
print("Start loss:", math.exp(start_eval['eval_loss']))

trainer_mlm.train()

final_eval = trainer_mlm.evaluate()
print("Final loss:", math.exp(final_eval['eval_loss']))

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Start loss: 4.123747560510189


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,1.117100,0.999623,0.002600
2,1.011300,0.909080,0.002600
3,0.955500,0.868303,0.002600


Final loss: 2.3871210083848338


In [21]:
model_tune = AutoModelForSequenceClassification.from_pretrained("distilbert/distilroberta-base", num_labels=2, ignore_mismatched_sizes=True)

model_tune.load_state_dict(trainer_mlm.model.state_dict(), strict=False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_IncompatibleKeys(missing_keys=['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias'], unexpected_keys=['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias'])

In [22]:
training_args_tune = TrainingArguments(
    output_dir="classifier_tune",
    eval_strategy="epoch",
    push_to_hub=False,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    metric_for_best_model="accuracy"
)

trainer_tune = Trainer(
    model=model_tune,
    args=training_args_tune,
    train_dataset=train,
    eval_dataset=valid,
    compute_metrics=compute_metrics,
)

trainer_tune.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.290200,0.193034,0.909574,0.911765
2,0.204000,0.224897,0.919326,0.920663
3,0.182400,0.185137,0.931738,0.935021


TrainOutput(global_step=1692, training_loss=0.21942710876464844, metrics={'train_runtime': 863.1361, 'train_samples_per_second': 15.682, 'train_steps_per_second': 1.96, 'total_flos': 1793078708207616.0, 'train_loss': 0.21942710876464844, 'epoch': 3.0})

In [23]:
pred_tune = trainer_tune.predict(valid)
logits_tune, labels_tune = pred_tune.predictions, pred_tune.label_ids
preds_tune = np.argmax(logits_tune, axis=-1)

acc_tune = accuracy_score(labels_tune, preds_tune)
f1_tune = f1_score(labels_tune, preds_tune)
cm_tune = confusion_matrix(labels_tune, preds_tune)

print("accuracy_score:", acc_tune)
print("f1_score:", f1_tune)
print("confusion_matrix:", cm_tune)

accuracy_score: 0.9317375886524822
f1_score: 0.9350210970464135
confusion_matrix: [[497  36]
 [ 41 554]]


Итоговый результат получился лучше чем при начальном обучении

In [24]:
submission = pd.read_csv("submission.csv", delimiter=",")

submission_dataset = Dataset.from_dict({"text": submission["text"].tolist()})

submission_dataset = submission_dataset.map(tokenize, batched=True)

preds_output = trainer_tune.predict(submission_dataset)
logits = preds_output.predictions
pred_labels = np.argmax(logits, axis=-1)

submission["label"] = pred_labels

submission.to_csv("submission_pred.csv", index=False)

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

In [25]:
submission_pred = pd.read_csv("submission_pred.csv", delimiter=",")

submission_pred.head()

,text,label
0,"Он говорил ему, указывая на поля,",1
1,"Тревога беспредметная и бесцельная в настоящем,",1
2,"- Я о тебе, третьего дня",1
3,- Что ж вы? - закричал,1
4,Он с мучением задавал себе этот,1
